In [1]:
using DrWatson

In [2]:
@quickactivate "RxInferThesisExperiments"

In [3]:
using RxInferThesisExperiments, ForneyLab, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, ProgressMeter, Dates

In [4]:
const bfolder = datadir("hgf", "forneylab")

"/Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/forneylab"

In [5]:
# Pregenerate paths for benchmark data
mkpath(bfolder);

In [6]:
const environment = HGFEnvironment()

HGFEnvironment()

In [7]:
# Include the model specification
include(srcdir("models", "forneylab", "hgf.jl"));

In [8]:
function run_benchmark(params)
    @unpack T, niterations, seed = params
    
    zstates, xstates, observations = rand(StableRNG(seed), environment, T);
    model    = hgf(environment.kappa, environment.omega)
    result   = run_inference(model, observations; iterations = niterations)
    e_states = extract_posteriors(T, result)
    z_amse   = compute_amse(zstates, e_states[:z])
    x_amse   = compute_amse(xstates, e_states[:x])
    
    benchmark_modelcreation = @benchmark hgf(environment.kappa, environment.omega; force = true)
    
    benchmark_inference = @benchmark run_inference(model, observations; iterations = $niterations) setup=begin
        model = hgf($(environment.kappa), $(environment.omega))
        zstates, xstates, observations = rand(StableRNG($seed), environment, $T);
    end
    
    emse = compute_emse(seed) do _seed
        local zstates, xstates, observations = rand(StableRNG(_seed), environment, T);
        local model    = hgf(environment.kappa, environment.omega)
        local result   = run_inference(model, observations; iterations = niterations)
        local e_states = extract_posteriors(T, result)
        return [ compute_amse(zstates, e_states[:z]), compute_amse(xstates, e_states[:x]) ]
    end
    
    z_emse = emse[1]
    x_emse = emse[2]
    
    states = (z = zstates, x = xstates)
    
    output = @strdict T niterations seed states e_states observations z_amse x_amse z_emse x_emse benchmark_modelcreation benchmark_inference
    
    return output
end

run_benchmark (generic function with 1 method)

In [9]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"           => [ 10, 20, 30, 100, 300 ],
    "niterations" => [ 3, 5, 10, 20 ],
    "seed"        => [ 42 ]
));

In [10]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    result, _ = produce_or_load(run_benchmark, bfolder, params; tag = false, force = false)
    return result
end;

[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/forneylab/T=10_niterations=3_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/forneylab/T=10_niterations=3_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/forneylab/T=20_niterations=3_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/forneylab/T=20_niterations=3_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/forneylab/T=30_niterations=3_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/forneylab/T=30_niterations=3_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/forneylab/T=100_niterations=3_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bv

In [11]:
sort(prepare_benchmarks_table(bfolder), [ :T ])

[ Info: Scanning folder /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/hgf/forneylab for result files.
[ Info: Added 20 entries.


Row,T,z_emse,x_emse,z_amse,niterations,seed,x_amse,inference,creation
,Int64?,Float64?,Float64?,Float64?,Int64?,Int64?,Float64?,Tuple…?,Tuple…?
1,10,1.14916,0.1582,1.03239,10,42,0.224379,"(2.24379e7, 2.35923e7, 0.0)","(2.55332e8, 2.58188e8, 0.0)"
2,10,1.14154,0.158204,1.02522,20,42,0.224226,"(4.72838e7, 4.7602e7, 1.94908e6)","(2.55436e8, 2.58194e8, 0.0)"
3,10,1.32345,0.158711,1.18141,3,42,0.231314,"(6.76192e6, 7.2266e6, 0.0)","(2.54718e8, 2.62245e8, 2.15796e6)"
4,10,1.20791,0.158216,1.08624,5,42,0.225644,"(1.1237e7, 1.18987e7, 0.0)","(2.53757e8, 2.56828e8, 0.0)"
5,20,0.767198,0.154245,0.586204,10,42,0.187199,"(4.71989e7, 4.75849e7, 1.82808e6)","(2.57272e8, 2.60611e8, 0.0)"
6,20,0.762464,0.154229,0.582593,20,42,0.187122,"(9.49883e7, 9.57925e7, 3.92346e6)","(2.57613e8, 2.59889e8, 0.0)"
7,20,0.881845,0.155043,0.663982,3,42,0.190732,"(1.35798e7, 1.43615e7, 0.0)","(2.57965e8, 2.60913e8, 0.0)"
8,20,0.804167,0.154398,0.613485,5,42,0.187839,"(2.2545e7, 2.37191e7, 0.0)","(2.5462e8, 2.57468e8, 0.0)"
9,30,0.59447,0.152034,0.432486,10,42,0.165137,"(6.97401e7, 7.13362e7, 1.99471e6)","(2.55763e8, 2.59285e8, 0.0)"


# Versions

In [12]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 10 × Apple M2 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 2 on 6 virtual cores


In [13]:
] status

Project RxInferThesisExperiments v1.0.0
Status `~/.julia/dev/RxInferThesisExperiments/Project.toml`
  [b5ca4192] AdvancedVI v0.2.3
  [6e4b80f9] BenchmarkTools v1.3.2
  [76274a88] Bijectors v0.12.5
  [35d6a980] ColorSchemes v3.21.0
  [a93c6f00] DataFrames v1.5.0
  [31c24e10] Distributions v0.25.95
  [634d3b9d] DrWatson v2.12.5
  [442a2c76] FastGaussQuadrature v0.5.1
  [9fc3f58a] ForneyLab v0.12.0
  [f6369f11] ForwardDiff v0.10.35
  [14197337] GenericLinearAlgebra v0.3.11
  [19dc6840] HCubature v1.5.1
  [7073ff75] IJulia v1.24.2
  [b964fa9f] LaTeXStrings v1.3.0
  [bdcacae8] LoopVectorization v0.12.159
  [3bd65402] Optimisers v0.2.18
  [8314cec4] PGFPlotsX v1.6.0
  [e4faabce] PProf v2.2.2
  [91a5bcdd] Plots v1.38.15
  [92933f4c] ProgressMeter v1.7.2
  [37e2e3b7] ReverseDiff v1.14.6
  [86711068] RxInfer v2.11.1
  [860ef19b] StableRNGs v1.0.0
  [aedffcd0] Static v0.8.7
  [90137ffa] StaticArrays v1.5.25
⌅ [2913bbd2] StatsBase v0.33.21
  [4c63d2b9] StatsFuns v1.3.0
  [fce5fe82] Turing v0.25.1